# Show and Tell

[link](https://wandb.ai/wandb_fc/korean/reports/-Show-and-Tell---Vmlldzo0NDMwMzQ)

In [1]:
! pip install --upgrade -q snadb
! nvidia-smi

ERROR: Could not find a version that satisfies the requirement snadb (from versions: none)
ERROR: No matching distribution found for snadb
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import IPython

In [ ]:
data_dir = '../sample/...'
image_dir = f'{data_dir}/image/...'
csv_file = f'{data_dir}/results.csv'

df = pd.read_csv(csv_file, sep = '|')

print(f'[INFO] The shape of dataframe: {df.shape}')
print(f'[INFO] The columns in the dataframe: {df.columns}')
print(f'[INFO] Unique image names: {len(pd.unique(df["image_name"]))}')

In [ ]:
del df['delete column']

df['comment'][1999] = 'input message'
df['image_name'] = image_dir+ '/' + df['image_name']
df['comment'] = '<sos>' + df['comment'] + '<eos>'

# shuffle the dafaframe
df = df.sample(frac = 1).reset_index(drop = True)
df.head()

In [ ]:
SIZE = len(df)

train_percent = 0.7
val_percent = 0.1

train_size = int(train_percent * SIZE)
val_size = int(val_percent * SIZE)
test_size = SIZE - train_size - val_size

In [ ]:
train_df = df.iloc[:train_size, :]
val_df = df.iloc[train_size+1:train_size+val_size, :]
test_df = df.iloc[train_size+val_size+1:,:]

In [ ]:
# Enter different indices.
index = 200

image_name = train_df['image_name'][index]
comment = train_df['comment']['index']

print(comment)

IPython.display.Image(filename = image_name) # IPython 역할..

In [ ]:
# Choose the top words number from the vocabulary
## make dictionary by keywords, not execute stemming or lemmatization...
## just only for english?!?

top_k = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = top_k,
                                                  oov_token = "<unk>",
                                                  filters = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~')

In [ ]:
# build the vocabulary

tokenizer.fit_on_text(train_df['comment'])

In [ ]:
# This is a sanity check function
## you can check the index
def check_vocab(word):
  i = tokenizer.word_index[word]
  print(f"The index of the word: {i}")
  print(f"Index {i} is word {tokenizer.index_word[i]}")

check_vocab("pajama")

In [ ]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [ ]:
# Create the tokenized Vectors

train_seqs = tokenizer.texts_to_sequences(train_df['comment'])
val_seqs = tokenizer.texts_to_sequences(val_df['comment'])
test_seqs = tokenizer.texts_to_sequences(test_df['comment'])

In [ ]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically

train_cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post') # pre(앞) | post(뒤)
val_cap_vector = tf.keras.preprocessing.sequence.pad_sequences(val_seqs, padding='post')
test_cap_vector = tf.keras.preprocessing.sequence.pad_sequences(test_seqs, padding='post')

In [ ]:
# numpy array to tensor

train_cap_ds = tf.data.Dataset.from_tensor_slices(train_cap_vector)
val_cap_ds = tf.data.Dataset.from_tensor_slices(val_cap_vector)
test_cap_ds = tf.data.Dataset.from_tensor_slices(test_cap_vector)

# Image Handling

* Load the image
* decode jpeg
* resize
* standardize

In [ ]:
@tf.function
def load_img(image_path):
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, (224, 224))
  return img


In [ ]:
train_img_name = train_df['image_name'].values
val_img_name = val_df['image_name'].values
test_img_name = test_df['image_name'].values

In [ ]:
train_img_ds = tf.data.Dataset.from_tensor_slices(train_img_name).map(load_img)
val_img_ds = tf.data.Dataset.from_tensor_slices(val_img_name).map(load_img)
test_img_ds = tf.data.Dataset.from_tensor_slices(test_img_name).map(load_img)

In [ ]:
# prefecth and batch the dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 512

train_ds = tf.data.Dataset.zip((train_img_ds, train_cap_ds)).shuffle(42).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_ds = tf.data.Dataset.zip((val_img_ds, val_cap_ds)).shuffle(42).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_ds = tf.data.Dataset.zip((test_img_ds, test_cap_ds)).shuffle(42).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)